In [1]:
#Import Hamilton County voter data file pulled from https://www6.sos.state.oh.us/ords/f?p=111:1
#I'm removing the 'LAST_NAME, 'FIRST_NAME', and 'MIDDLE_NAME' fields off the top because I end up calling '.head()'
#a lot & even though this is public record (and I generally agree with it being so)
#it lives in a .csv file & there's no reason to single out the first 5 people over&over again
#I would like to eventually feed some of this data through one of the algorithms 
#which predicts sex & race/ethnicity based on first/last names & census block/track 
#but that will be a longer term project as all of the APIs that offer this are pretty expensive 
#but it looks like you can train your own models from publicly available data - but that's a bigger project
import pandas as pd
hc_full = pd.read_csv('HAMILTON.txt')
drop_columns = ['LAST_NAME', 'FIRST_NAME', 'MIDDLE_NAME', 'SUFFIX']
hc_full=hc_full.drop(columns=drop_columns)
hc_full.head()

C:\Users\Eliza\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (58,64,66,67,73,74,78,79,82,87,93,96,97,100,103) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SOS_VOTERID,COUNTY_NUMBER,COUNTY_ID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,PARTY_AFFILIATION,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,...,GENERAL-06/07/2016,PRIMARY-09/13/2016,GENERAL-11/08/2016,PRIMARY-05/02/2017,PRIMARY-09/12/2017,GENERAL-11/07/2017,PRIMARY-05/08/2018,GENERAL-08/07/2018,GENERAL-11/06/2018,PRIMARY-05/07/2019
0,OH0013910067,31,822992,1963-07-06,1989-07-17,ACTIVE,NaN,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,...,NaN,NaN,X,X,NaN,NaN,NaN,NaN,X,NaN
1,OH0013638654,31,162906,1938-08-26,1989-07-17,ACTIVE,NaN,1448 FIELDCREST CT,NaN,CINCINNATI,...,NaN,NaN,X,NaN,NaN,X,NaN,NaN,NaN,NaN
2,OH0013744969,31,1139049,1975-10-09,1993-09-08,ACTIVE,R,7206 MIAMI HILLS DR,NaN,CINCINNATI,...,NaN,NaN,X,NaN,NaN,NaN,NaN,NaN,X,NaN
3,OH0020648383,31,1689793,1968-01-01,2009-05-05,ACTIVE,NaN,11606 HANOVER RD,NaN,CINCINNATI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN
4,OH0021001272,31,1705121,1991-06-18,2010-03-04,ACTIVE,R,7822 MITCHELL PARK DR,NaN,CLEVES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN


In [2]:
list(hc_full.columns)

['SOS_VOTERID',
 'COUNTY_NUMBER',
 'COUNTY_ID',
 'DATE_OF_BIRTH',
 'REGISTRATION_DATE',
 'VOTER_STATUS',
 'PARTY_AFFILIATION',
 'RESIDENTIAL_ADDRESS1',
 'RESIDENTIAL_SECONDARY_ADDR',
 'RESIDENTIAL_CITY',
 'RESIDENTIAL_STATE',
 'RESIDENTIAL_ZIP',
 'RESIDENTIAL_ZIP_PLUS4',
 'RESIDENTIAL_COUNTRY',
 'RESIDENTIAL_POSTALCODE',
 'MAILING_ADDRESS1',
 'MAILING_SECONDARY_ADDRESS',
 'MAILING_CITY',
 'MAILING_STATE',
 'MAILING_ZIP',
 'MAILING_ZIP_PLUS4',
 'MAILING_COUNTRY',
 'MAILING_POSTAL_CODE',
 'CAREER_CENTER',
 'CITY',
 'CITY_SCHOOL_DISTRICT',
 'COUNTY_COURT_DISTRICT',
 'CONGRESSIONAL_DISTRICT',
 'COURT_OF_APPEALS',
 'EDU_SERVICE_CENTER_DISTRICT',
 'EXEMPTED_VILL_SCHOOL_DISTRICT',
 'LIBRARY',
 'LOCAL_SCHOOL_DISTRICT',
 'MUNICIPAL_COURT_DISTRICT',
 'PRECINCT_NAME',
 'PRECINCT_CODE',
 'STATE_BOARD_OF_EDUCATION',
 'STATE_REPRESENTATIVE_DISTRICT',
 'STATE_SENATE_DISTRICT',
 'TOWNSHIP',
 'VILLAGE',
 'WARD',
 'PRIMARY-03/07/2000',
 'GENERAL-11/07/2000',
 'SPECIAL-05/08/2001',
 'GENERAL-11/06/2001',

In [3]:
#Since this is a subset of the statwide data file, there are some election dates that most current Hamilton county 
#residents would not have participated in -i.e., most 'Special' elections which are frequently local school district tax leavies
#& their associated primaries 
#Count the number of NaN entries for each column & percent of total
def missing_table(df):
        mis_val = df.isna().sum()
        mis_val_percent = 100 * df.isna().sum() / len(df)
        mz_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

mt = missing_table(hc_full)
mt

Your dataframe has 102 columns and 574834 Rows.
There are 84 columns that have missing values.


,Missing Values,% of Total Values,Data Type
COUNTY_COURT_DISTRICT,574834,100.0,float64
PRIMARY-10/14/2008,574834,100.0,float64
RESIDENTIAL_COUNTRY,574834,100.0,float64
RESIDENTIAL_POSTALCODE,574834,100.0,float64
GENERAL-11/18/2008,574834,100.0,float64
MAILING_SECONDARY_ADDRESS,574834,100.0,float64
PRIMARY-09/29/2009,574834,100.0,float64
MAILING_COUNTRY,574834,100.0,float64
MAILING_POSTAL_CODE,574834,100.0,float64
LIBRARY,574834,100.0,float64


In [4]:
#Review columns that are missing high percentages of values
mt_reset = mt.reset_index()
mt_reset.loc[mt_reset['% of Total Values']>90]

,index,Missing Values,% of Total Values,Data Type
0,COUNTY_COURT_DISTRICT,574834,100.0,float64
1,PRIMARY-10/14/2008,574834,100.0,float64
2,RESIDENTIAL_COUNTRY,574834,100.0,float64
3,RESIDENTIAL_POSTALCODE,574834,100.0,float64
4,GENERAL-11/18/2008,574834,100.0,float64
5,MAILING_SECONDARY_ADDRESS,574834,100.0,float64
6,PRIMARY-09/29/2009,574834,100.0,float64
7,MAILING_COUNTRY,574834,100.0,float64
8,MAILING_POSTAL_CODE,574834,100.0,float64
9,LIBRARY,574834,100.0,float64


In [5]:
#Drop all columns where >99.9% of the rows are missing values
mlist = mt_reset.loc[mt_reset['% of Total Values']>99.9]['index']
hc_working = hc_full.drop(columns = mlist)
hc_working.columns

Index(['SOS_VOTERID', 'COUNTY_NUMBER', 'COUNTY_ID', 'DATE_OF_BIRTH',
       'REGISTRATION_DATE', 'VOTER_STATUS', 'PARTY_AFFILIATION',
       'RESIDENTIAL_ADDRESS1', 'RESIDENTIAL_SECONDARY_ADDR',
       'RESIDENTIAL_CITY', 'RESIDENTIAL_STATE', 'RESIDENTIAL_ZIP',
       'RESIDENTIAL_ZIP_PLUS4', 'MAILING_ADDRESS1', 'MAILING_CITY',
       'MAILING_STATE', 'MAILING_ZIP', 'CAREER_CENTER', 'CITY',
       'CITY_SCHOOL_DISTRICT', 'CONGRESSIONAL_DISTRICT', 'COURT_OF_APPEALS',
       'EDU_SERVICE_CENTER_DISTRICT', 'EXEMPTED_VILL_SCHOOL_DISTRICT',
       'LOCAL_SCHOOL_DISTRICT', 'MUNICIPAL_COURT_DISTRICT', 'PRECINCT_NAME',
       'PRECINCT_CODE', 'STATE_BOARD_OF_EDUCATION',
       'STATE_REPRESENTATIVE_DISTRICT', 'STATE_SENATE_DISTRICT', 'TOWNSHIP',
       'VILLAGE', 'WARD', 'PRIMARY-03/07/2000', 'GENERAL-11/07/2000',
       'SPECIAL-05/08/2001', 'GENERAL-11/06/2001', 'PRIMARY-05/07/2002',
       'GENERAL-11/05/2002', 'SPECIAL-05/06/2003', 'GENERAL-11/04/2003',
       'PRIMARY-03/02/2004', 'GENERA

In [6]:
#Per the Voter File Layout doc on the SOS website, the 'SOS Voter ID' is a unique identifier
#so I could easily reconect these fields later if wanted, but for now removing unwanted fields
#per the documentation 'PARTY_AFFILIATION' is 
#"The voter’s party affiliation as recorded from their last Primary Election history"
#so also removing this field and will atempt to create a proxy for party
remove_list = ['COUNTY_NUMBER', 'COUNTY_ID', 'PARTY_AFFILIATION', 'MAILING_ADDRESS1',
       'MAILING_CITY', 'MAILING_STATE', 'MAILING_ZIP', 'CAREER_CENTER', 'CITY',
       'CITY_SCHOOL_DISTRICT', 'CONGRESSIONAL_DISTRICT', 'COURT_OF_APPEALS',
       'EDU_SERVICE_CENTER_DISTRICT', 'EXEMPTED_VILL_SCHOOL_DISTRICT',
       'LOCAL_SCHOOL_DISTRICT', 'MUNICIPAL_COURT_DISTRICT', 'STATE_BOARD_OF_EDUCATION',
       'STATE_REPRESENTATIVE_DISTRICT', 'STATE_SENATE_DISTRICT', 'TOWNSHIP',
       'VILLAGE', 'WARD']
hc_working = hc_working.drop(columns =remove_list)

In [7]:
hc_working.head()

,SOS_VOTERID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,...,GENERAL-11/04/2014,PRIMARY-05/05/2015,GENERAL-11/03/2015,PRIMARY-03/15/2016,GENERAL-11/08/2016,PRIMARY-05/02/2017,GENERAL-11/07/2017,PRIMARY-05/08/2018,GENERAL-11/06/2018,PRIMARY-05/07/2019
0,OH0013910067,1963-07-06,1989-07-17,ACTIVE,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,OH,45233,NaN,...,X,NaN,X,NaN,X,X,NaN,NaN,X,NaN
1,OH0013638654,1938-08-26,1989-07-17,ACTIVE,1448 FIELDCREST CT,NaN,CINCINNATI,OH,45231,NaN,...,X,NaN,NaN,NaN,X,NaN,X,NaN,NaN,NaN
2,OH0013744969,1975-10-09,1993-09-08,ACTIVE,7206 MIAMI HILLS DR,NaN,CINCINNATI,OH,45243,NaN,...,X,NaN,NaN,R,X,NaN,NaN,NaN,X,NaN
3,OH0020648383,1968-01-01,2009-05-05,ACTIVE,11606 HANOVER RD,NaN,CINCINNATI,OH,45240,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,NaN
4,OH0021001272,1991-06-18,2010-03-04,ACTIVE,7822 MITCHELL PARK DR,NaN,CLEVES,OH,45002,NaN,...,NaN,NaN,X,R,NaN,NaN,NaN,NaN,X,NaN


In [8]:
hc_working.columns

Index(['SOS_VOTERID', 'DATE_OF_BIRTH', 'REGISTRATION_DATE', 'VOTER_STATUS',
       'RESIDENTIAL_ADDRESS1', 'RESIDENTIAL_SECONDARY_ADDR',
       'RESIDENTIAL_CITY', 'RESIDENTIAL_STATE', 'RESIDENTIAL_ZIP',
       'RESIDENTIAL_ZIP_PLUS4', 'PRECINCT_NAME', 'PRECINCT_CODE',
       'PRIMARY-03/07/2000', 'GENERAL-11/07/2000', 'SPECIAL-05/08/2001',
       'GENERAL-11/06/2001', 'PRIMARY-05/07/2002', 'GENERAL-11/05/2002',
       'SPECIAL-05/06/2003', 'GENERAL-11/04/2003', 'PRIMARY-03/02/2004',
       'GENERAL-11/02/2004', 'SPECIAL-02/08/2005', 'PRIMARY-05/03/2005',
       'GENERAL-11/08/2005', 'SPECIAL-02/07/2006', 'PRIMARY-05/02/2006',
       'GENERAL-11/07/2006', 'PRIMARY-05/08/2007', 'GENERAL-11/06/2007',
       'PRIMARY-03/04/2008', 'GENERAL-11/04/2008', 'PRIMARY-05/05/2009',
       'GENERAL-11/03/2009', 'PRIMARY-05/04/2010', 'GENERAL-11/02/2010',
       'PRIMARY-05/03/2011', 'GENERAL-11/08/2011', 'PRIMARY-03/06/2012',
       'GENERAL-11/06/2012', 'PRIMARY-05/07/2013', 'GENERAL-11/05/2013',


In [9]:
#I'm dropping 'off-year' races (non US House years)
#These races have very low turnout and would probably be worth studying on their own
#(muncipal/state courts & school board are important but not big vote getters)
#OH's State Execuive offices, State Senate, and State House races also all fall on even years
#& Hamilton county has not had a significant special election recently - the 2006 special is all bond issues
remove_list2 = ['SPECIAL-05/08/2001', 'GENERAL-11/06/2001', 
                'SPECIAL-05/06/2003', 'GENERAL-11/04/2003',
                'SPECIAL-02/08/2005', 'PRIMARY-05/03/2005', 'GENERAL-11/08/2005', 
                'SPECIAL-02/07/2006',
                'PRIMARY-05/08/2007', 'GENERAL-11/06/2007',
                'PRIMARY-05/05/2009', 'GENERAL-11/03/2009', 
                'PRIMARY-05/03/2011', 'GENERAL-11/08/2011',
                'PRIMARY-05/07/2013', 'GENERAL-11/05/2013', 
                'PRIMARY-05/05/2015', 'GENERAL-11/03/2015', 
                'PRIMARY-05/02/2017', 'GENERAL-11/07/2017',
                'PRIMARY-05/07/2019']
hc_working2 = hc_working.drop(columns =remove_list2)
hc_working2.head()

,SOS_VOTERID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,...,PRIMARY-05/04/2010,GENERAL-11/02/2010,PRIMARY-03/06/2012,GENERAL-11/06/2012,PRIMARY-05/06/2014,GENERAL-11/04/2014,PRIMARY-03/15/2016,GENERAL-11/08/2016,PRIMARY-05/08/2018,GENERAL-11/06/2018
0,OH0013910067,1963-07-06,1989-07-17,ACTIVE,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,OH,45233,NaN,...,X,X,NaN,X,NaN,X,NaN,X,NaN,X
1,OH0013638654,1938-08-26,1989-07-17,ACTIVE,1448 FIELDCREST CT,NaN,CINCINNATI,OH,45231,NaN,...,NaN,X,NaN,X,NaN,X,NaN,X,NaN,NaN
2,OH0013744969,1975-10-09,1993-09-08,ACTIVE,7206 MIAMI HILLS DR,NaN,CINCINNATI,OH,45243,NaN,...,NaN,X,NaN,X,NaN,X,R,X,NaN,X
3,OH0020648383,1968-01-01,2009-05-05,ACTIVE,11606 HANOVER RD,NaN,CINCINNATI,OH,45240,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
4,OH0021001272,1991-06-18,2010-03-04,ACTIVE,7822 MITCHELL PARK DR,NaN,CLEVES,OH,45002,NaN,...,X,NaN,NaN,X,NaN,NaN,R,NaN,NaN,X


In [10]:
hc_working2.columns

Index(['SOS_VOTERID', 'DATE_OF_BIRTH', 'REGISTRATION_DATE', 'VOTER_STATUS',
       'RESIDENTIAL_ADDRESS1', 'RESIDENTIAL_SECONDARY_ADDR',
       'RESIDENTIAL_CITY', 'RESIDENTIAL_STATE', 'RESIDENTIAL_ZIP',
       'RESIDENTIAL_ZIP_PLUS4', 'PRECINCT_NAME', 'PRECINCT_CODE',
       'PRIMARY-03/07/2000', 'GENERAL-11/07/2000', 'PRIMARY-05/07/2002',
       'GENERAL-11/05/2002', 'PRIMARY-03/02/2004', 'GENERAL-11/02/2004',
       'PRIMARY-05/02/2006', 'GENERAL-11/07/2006', 'PRIMARY-03/04/2008',
       'GENERAL-11/04/2008', 'PRIMARY-05/04/2010', 'GENERAL-11/02/2010',
       'PRIMARY-03/06/2012', 'GENERAL-11/06/2012', 'PRIMARY-05/06/2014',
       'GENERAL-11/04/2014', 'PRIMARY-03/15/2016', 'GENERAL-11/08/2016',
       'PRIMARY-05/08/2018', 'GENERAL-11/06/2018'],
      dtype='object')

In [11]:
#Per provided SOS documentation, each election column is filled with the party the voter selected a ballot for
#C - Constitution Party
#D - Democrat Party
#E - Reform Party
#G - Green Party
#L - Libertarian Party
#N - Natural Law Party
#R - Republican Party
#S - Socialist Party 
#X - Voted without declaring party affiliation
#Blank - Indicates that there is no voting record for this voter for this election
#
#
#Voters do not select a ballot type for general elections, only primaries, so are should be type X
#Primary voters who select type X are not able to vote in a lot of races 
#(typically only judicial & other local non-partisan races - school board, city council, etc.)
#All the blanks = NaN
election_list = ['PRIMARY-03/07/2000', 'GENERAL-11/07/2000',
       'PRIMARY-05/07/2002', 'GENERAL-11/05/2002', 'PRIMARY-03/02/2004',
       'GENERAL-11/02/2004', 'PRIMARY-05/02/2006', 'GENERAL-11/07/2006',
       'PRIMARY-03/04/2008', 'GENERAL-11/04/2008', 'PRIMARY-05/04/2010',
       'GENERAL-11/02/2010', 'PRIMARY-03/06/2012', 'GENERAL-11/06/2012',
       'PRIMARY-05/06/2014', 'GENERAL-11/04/2014', 'PRIMARY-03/15/2016',
       'GENERAL-11/08/2016', 'PRIMARY-05/08/2018', 'GENERAL-11/06/2018']
hc_working2[election_list].apply(pd.Series.value_counts)

,PRIMARY-03/07/2000,GENERAL-11/07/2000,PRIMARY-05/07/2002,GENERAL-11/05/2002,PRIMARY-03/02/2004,GENERAL-11/02/2004,PRIMARY-05/02/2006,GENERAL-11/07/2006,PRIMARY-03/04/2008,GENERAL-11/04/2008,PRIMARY-05/04/2010,GENERAL-11/02/2010,PRIMARY-03/06/2012,GENERAL-11/06/2012,PRIMARY-05/06/2014,GENERAL-11/04/2014,PRIMARY-03/15/2016,GENERAL-11/08/2016,PRIMARY-05/08/2018,GENERAL-11/06/2018
C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
D,28513.0,NaN,11324.0,NaN,36377.0,NaN,23020.0,NaN,126386.0,NaN,29895.0,NaN,19945.0,NaN,21838.0,NaN,97311.0,NaN,51958.0,NaN
G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.0,NaN,126.0,NaN,137.0,NaN,443.0,NaN,550.0,NaN
L,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,280.0,NaN,205.0,NaN,364.0,NaN,NaN,NaN,NaN,NaN
N,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R,59225.0,NaN,19054.0,NaN,40954.0,NaN,41959.0,3.0,61510.0,NaN,45307.0,NaN,69133.0,NaN,34412.0,NaN,127111.0,NaN,47915.0,NaN
S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
X,8001.0,211838.0,12618.0,153513.0,14608.0,282273.0,10650.0,213272.0,7916.0,329976.0,12033.0,237490.0,4334.0,367358.0,8942.0,223734.0,670.0,399846.0,9804.0,340738.0


In [12]:
#I'm going to rename the election columns to be more uniform & easier/shorter to work with
#will create dictionary to store the 'proper' name & date if needed later
election_dict = {'PRIMARY-03/07/2000':'P_00', 'GENERAL-11/07/2000':'G_00',
                'PRIMARY-05/07/2002':'P_02', 'GENERAL-11/05/2002':'G_02',
                'PRIMARY-03/02/2004':'P_04', 'GENERAL-11/02/2004':'G_04',
                'PRIMARY-05/02/2006':'P_06', 'GENERAL-11/07/2006':'G_06',
                'PRIMARY-03/04/2008':'P_08', 'GENERAL-11/04/2008':'G_08',
                'PRIMARY-05/04/2010':'P_10', 'GENERAL-11/02/2010':'G_10', 
                'PRIMARY-03/06/2012':'P_12', 'GENERAL-11/06/2012':'G_12',
                'PRIMARY-05/06/2014':'P_14', 'GENERAL-11/04/2014':'G_14',
                'PRIMARY-03/15/2016':'P_16', 'GENERAL-11/08/2016':'G_16',
                'PRIMARY-05/08/2018':'P_18', 'GENERAL-11/06/2018':'G_18'}
hc_working2=hc_working2.rename(columns = election_dict)
hc_working2.head()

,SOS_VOTERID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,...,P_10,G_10,P_12,G_12,P_14,G_14,P_16,G_16,P_18,G_18
0,OH0013910067,1963-07-06,1989-07-17,ACTIVE,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,OH,45233,NaN,...,X,X,NaN,X,NaN,X,NaN,X,NaN,X
1,OH0013638654,1938-08-26,1989-07-17,ACTIVE,1448 FIELDCREST CT,NaN,CINCINNATI,OH,45231,NaN,...,NaN,X,NaN,X,NaN,X,NaN,X,NaN,NaN
2,OH0013744969,1975-10-09,1993-09-08,ACTIVE,7206 MIAMI HILLS DR,NaN,CINCINNATI,OH,45243,NaN,...,NaN,X,NaN,X,NaN,X,R,X,NaN,X
3,OH0020648383,1968-01-01,2009-05-05,ACTIVE,11606 HANOVER RD,NaN,CINCINNATI,OH,45240,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X
4,OH0021001272,1991-06-18,2010-03-04,ACTIVE,7822 MITCHELL PARK DR,NaN,CLEVES,OH,45002,NaN,...,X,NaN,NaN,X,NaN,NaN,R,NaN,NaN,X


In [13]:
#Creating list of primary elections & general elections in case needed later
#want to quickly look at primary election ballot selctions as it looks like 3rd party selections are increasing
primary_list = ['P_00', 'P_02', 'P_04', 'P_06', 'P_08', 'P_10', 'P_12', 'P_14', 'P_16', 'P_18']
general_list = ['G_00', 'G_02', 'G_04', 'G_06', 'G_08', 'G_10', 'G_12', 'G_14', 'G_16', 'G_18']
hc_working2[primary_list].apply(pd.Series.value_counts)

,P_00,P_02,P_04,P_06,P_08,P_10,P_12,P_14,P_16,P_18
C,NaN,NaN,NaN,NaN,NaN,100.0,1.0,1.0,NaN,NaN
D,28513.0,11324.0,36377.0,23020.0,126386.0,29895.0,19945.0,21838.0,97311.0,51958.0
G,NaN,NaN,NaN,NaN,NaN,61.0,126.0,137.0,443.0,550.0
L,1.0,NaN,NaN,NaN,NaN,280.0,205.0,364.0,NaN,NaN
N,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R,59225.0,19054.0,40954.0,41959.0,61510.0,45307.0,69133.0,34412.0,127111.0,47915.0
S,NaN,NaN,NaN,NaN,NaN,39.0,NaN,NaN,NaN,NaN
X,8001.0,12618.0,14608.0,10650.0,7916.0,12033.0,4334.0,8942.0,670.0,9804.0


In [14]:
hc_working2[general_list].apply(pd.Series.value_counts)

,G_00,G_02,G_04,G_06,G_08,G_10,G_12,G_14,G_16,G_18
R,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN
X,211838.0,153513.0,282273.0,213272,329976.0,237490.0,367358.0,223734.0,399846.0,340738.0


In [15]:
#So the 'R' ballots on the G_06 election are highly unexpected - 
#I'm tempted to think this is an error, I have an email to the SOS office, but it will probably be a bit until I hear back
#I'm also noticing that voters appear to have voting history prior to their registration date,
#so I think registration date may update if the voter moves or changes any details of their registration - 
#so this may not be a super useful field
hc_working2.loc[hc_working2['G_06']=='R']

,SOS_VOTERID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,...,P_10,G_10,P_12,G_12,P_14,G_14,P_16,G_16,P_18,G_18
40416,OH0024604758,1955-08-03,2018-04-20,ACTIVE,117 HARTER AVE,NaN,CINCINNATI,OH,45246,NaN,...,NaN,X,NaN,X,NaN,NaN,R,X,NaN,X
59052,OH0023618408,1988-03-11,2016-03-14,ACTIVE,9213 DEERCROSS PKWY,APT 2A,CINCINNATI,OH,45236,NaN,...,NaN,X,NaN,X,NaN,X,NaN,X,NaN,X
273701,OH0025083046,1944-06-18,2019-05-07,ACTIVE,11130 SPRINGFIELD PIKE,APT B424,CINCINNATI,OH,45246,NaN,...,NaN,X,NaN,X,NaN,X,D,NaN,NaN,NaN


In [16]:
#documentation doesn't provide any insight into what 'VOTER_STATUS' is ('Shows current Voter Status')
#I know when you register in OH they send you a postcard - I think this is for address verification
#but 57k+ is pretty high - worth asking SOS what this means 
hc_working2['VOTER_STATUS'].value_counts()

ACTIVE          516841
CONFIRMATION     57993
Name: VOTER_STATUS, dtype: int64

In [17]:
#Going to create a few fields - 
#age, and %Democrat primary votes, %Republican primary votes, %3rd party primary votes
from datetime import date
today = date.today()
hc_working2['AGE']=pd.to_datetime(today)-pd.to_datetime(hc_working2['DATE_OF_BIRTH'])

In [18]:
hc_working2.head()

,SOS_VOTERID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,...,G_10,P_12,G_12,P_14,G_14,P_16,G_16,P_18,G_18,AGE
0,OH0013910067,1963-07-06,1989-07-17,ACTIVE,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,OH,45233,NaN,...,X,NaN,X,NaN,X,NaN,X,NaN,X,20545 days
1,OH0013638654,1938-08-26,1989-07-17,ACTIVE,1448 FIELDCREST CT,NaN,CINCINNATI,OH,45231,NaN,...,X,NaN,X,NaN,X,NaN,X,NaN,NaN,29625 days
2,OH0013744969,1975-10-09,1993-09-08,ACTIVE,7206 MIAMI HILLS DR,NaN,CINCINNATI,OH,45243,NaN,...,X,NaN,X,NaN,X,R,X,NaN,X,16067 days
3,OH0020648383,1968-01-01,2009-05-05,ACTIVE,11606 HANOVER RD,NaN,CINCINNATI,OH,45240,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,X,18905 days
4,OH0021001272,1991-06-18,2010-03-04,ACTIVE,7822 MITCHELL PARK DR,NaN,CLEVES,OH,45002,NaN,...,NaN,NaN,X,NaN,NaN,R,NaN,NaN,X,10336 days


In [19]:
primary_fields = primary_list + ['SOS_VOTERID']
primary_long = pd.wide_to_long(hc_working2[primary_fields], stubnames='P', i='SOS_VOTERID', j='YEAR', sep='_')
primary_long.head()

,,P
SOS_VOTERID,YEAR,
OH0013910067,0,NaN
OH0013638654,0,NaN
OH0013744969,0,R
OH0020648383,0,NaN
OH0021001272,0,NaN


In [20]:
primary_long = primary_long.dropna()
primary_long = primary_long.reset_index()
primary_long.head()

,SOS_VOTERID,YEAR,P
0,OH0013744969,0,R
1,OH0013574438,0,R
2,OH0013695360,0,R
3,OH0013744682,0,R
4,OH0013623418,0,D


In [21]:
primary_long['YEAR'].astype(str)
primary_long[['YEAR']].apply(pd.Series.value_counts)
#Did a quick check to the SOS website to make sure this is in line with actual turnout - it is
#Worth mentioning - b/c of how the data is provided 
#it reflects the voting histories of everyone currently registered in Hamilton county
#so as you go back in time you are going to be missing everyone who moved/died/was purged due to inactivity
#also, most states allow for certian people to not be included in these listings 
#I know OH & KY do not include voters who have been victims of domestic violence 
#or people who have active restraining orders since having their address public could put them in danger

,YEAR
16,225535
8,195812
18,110227
0,95741
12,93744
4,91939
10,87715
6,75629
14,65694
2,42996


In [22]:
#To approximate party affiliation I am going to take the percentage each primary voter selected either:
#an R ballot, a D ballot, a third party ballot = 
#(C, E, G, L, N, S - Consitution, Reform, Green Libertarian, Natural Law, Socialist), or unaffiliated (X)
#I am making the decision to lump 3rd parties together rather arbitrarily
#but given the small number of people who pick these & the general lack of primary races in 3rd parties
#I think it's ok

third_party = ['C', 'E', 'G', 'L', 'N', 'S']
primary_long['IS_R'] = [1 if x =='R' else 0 for x in primary_long['P']]
primary_long['IS_D'] = [1 if x =='D' else 0 for x in primary_long['P']]
primary_long['IS_X'] = [1 if x =='X' else 0 for x in primary_long['P']]
primary_long['IS_T'] = [1 if x in third_party else 0 for x in primary_long['P']]

In [23]:
primary_long.head()

,SOS_VOTERID,YEAR,P,IS_R,IS_D,IS_X,IS_T
0,OH0013744969,0,R,1,0,0,0
1,OH0013574438,0,R,1,0,0,0
2,OH0013695360,0,R,1,0,0,0
3,OH0013744682,0,R,1,0,0,0
4,OH0013623418,0,D,0,1,0,0


In [24]:
primary_history = primary_long.groupby('SOS_VOTERID').agg({'YEAR': 'count', 'IS_R': 'sum', 'IS_D':'sum', 
                                                          'IS_X': 'sum', 'IS_T': 'sum'})

In [25]:
primary_history.head()

,YEAR,IS_R,IS_D,IS_X,IS_T
SOS_VOTERID,,,,,
OH0011231100,2,0,2,0,0
OH0013538141,3,0,3,0,0
OH0013538145,3,1,2,0,0
OH0013538148,1,0,0,1,0
OH0013538149,6,0,6,0,0


In [26]:
primary_history['PER_R'] = primary_history['IS_R']/primary_history['YEAR']
primary_history['PER_D'] = primary_history['IS_D']/primary_history['YEAR']
primary_history['PER_X'] = primary_history['IS_X']/primary_history['YEAR']
primary_history['PER_T'] = primary_history['IS_T']/primary_history['YEAR']
primary_history.columns = primary_history.columns.get_level_values(0)
primary_history = primary_history.reset_index()
primary_history.head()
#This is a little crude as a metric -would probably want to do some kind of weighting in favor of more recent votes
#also - the 5th voter below is 6/6 D - which I think is probably a stronger indicator than a 1/1 D voter
#Probably lots of interesting things to look at as far as individual trends, but this is a start

,SOS_VOTERID,YEAR,IS_R,IS_D,IS_X,IS_T,PER_R,PER_D,PER_X,PER_T
0,OH0011231100,2,0,2,0,0,0.000000,1.000000,0.0,0.0
1,OH0013538141,3,0,3,0,0,0.000000,1.000000,0.0,0.0
2,OH0013538145,3,1,2,0,0,0.333333,0.666667,0.0,0.0
3,OH0013538148,1,0,0,1,0,0.000000,0.000000,1.0,0.0
4,OH0013538149,6,0,6,0,0,0.000000,1.000000,0.0,0.0


In [27]:
primary_percents = primary_history[['SOS_VOTERID', 'PER_R', 'PER_D','PER_X', 'PER_T']]
hc_working2=hc_working2.set_index('SOS_VOTERID')
primary_percents=primary_percents.set_index('SOS_VOTERID')
hc_working3 = hc_working2.join(primary_percents, how='left')
hc_working3.columns= hc_working3.columns.get_level_values(0)

,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,PRECINCT_NAME,...,G_14,P_16,G_16,P_18,G_18,AGE,PER_R,PER_D,PER_X,PER_T
SOS_VOTERID,,,,,,,,,,,,,,,,,,,,,
OH0013910067,1963-07-06,1989-07-17,ACTIVE,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,OH,45233,NaN,GREEN W,...,X,NaN,X,NaN,X,20545 days,0.0,0.0,1.0,0.0
OH0013638654,1938-08-26,1989-07-17,ACTIVE,1448 FIELDCREST CT,NaN,CINCINNATI,OH,45231,NaN,SPRINGFIELD H,...,X,NaN,X,NaN,NaN,29625 days,NaN,NaN,NaN,NaN
OH0013744969,1975-10-09,1993-09-08,ACTIVE,7206 MIAMI HILLS DR,NaN,CINCINNATI,OH,45243,NaN,MADEIRA D,...,X,R,X,NaN,X,16067 days,1.0,0.0,0.0,0.0
OH0020648383,1968-01-01,2009-05-05,ACTIVE,11606 HANOVER RD,NaN,CINCINNATI,OH,45240,NaN,FOREST PARK D,...,NaN,NaN,NaN,NaN,X,18905 days,NaN,NaN,NaN,NaN
OH0021001272,1991-06-18,2010-03-04,ACTIVE,7822 MITCHELL PARK DR,NaN,CLEVES,OH,45002,NaN,MIAMI TWP G,...,NaN,R,NaN,NaN,X,10336 days,0.5,0.0,0.5,0.0


In [28]:
hc_working3 = hc_working3.reset_index()
hc_working3.head()

,SOS_VOTERID,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,...,G_14,P_16,G_16,P_18,G_18,AGE,PER_R,PER_D,PER_X,PER_T
0,OH0013910067,1963-07-06,1989-07-17,ACTIVE,1672 DEVILS BACKBONE RD,NaN,CINCINNATI,OH,45233,NaN,...,X,NaN,X,NaN,X,20545 days,0.0,0.0,1.0,0.0
1,OH0013638654,1938-08-26,1989-07-17,ACTIVE,1448 FIELDCREST CT,NaN,CINCINNATI,OH,45231,NaN,...,X,NaN,X,NaN,NaN,29625 days,NaN,NaN,NaN,NaN
2,OH0013744969,1975-10-09,1993-09-08,ACTIVE,7206 MIAMI HILLS DR,NaN,CINCINNATI,OH,45243,NaN,...,X,R,X,NaN,X,16067 days,1.0,0.0,0.0,0.0
3,OH0020648383,1968-01-01,2009-05-05,ACTIVE,11606 HANOVER RD,NaN,CINCINNATI,OH,45240,NaN,...,NaN,NaN,NaN,NaN,X,18905 days,NaN,NaN,NaN,NaN
4,OH0021001272,1991-06-18,2010-03-04,ACTIVE,7822 MITCHELL PARK DR,NaN,CLEVES,OH,45002,NaN,...,NaN,R,NaN,NaN,X,10336 days,0.5,0.0,0.5,0.0


In [29]:
#Export data as csv
hc_working3.to_csv('hc_workingdata.csv',index=False)

In [42]:
#I also want to look at the typical percentages of people who vote in the primary but do not vote in the general
#In order to tell if 2016 was atypical
pg_fields = primary_list + general_list + ['SOS_VOTERID']
pg_long = pd.wide_to_long(hc_working3[pg_fields], stubnames=['P','G'], i='SOS_VOTERID', j='YEAR', sep='_')
pg_long.head()

,,P,G
SOS_VOTERID,YEAR,,
OH0013910067,0,NaN,X
OH0013638654,0,NaN,X
OH0013744969,0,R,X
OH0020648383,0,NaN,NaN
OH0021001272,0,NaN,NaN


In [43]:
pg_long = pg_long.reset_index()
pg_long.head()

,SOS_VOTERID,YEAR,P,G
0,OH0013910067,0,NaN,X
1,OH0013638654,0,NaN,X
2,OH0013744969,0,R,X
3,OH0020648383,0,NaN,NaN
4,OH0021001272,0,NaN,NaN


In [45]:
#Export data as csv
pg_long.to_csv('voting_hist.csv',index=False)